In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf


from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.specs import array_spec

In [ ]:
# Dummy variable
stocks=['JNJ', 'KO', 'T']
number_of_feat = 10

In [19]:
class PortfolioEnv(py_environment.PyEnvironment):
    def __init__(self):
        # Action: Portfolio ratio in stocks and cash
        self.action_spec = array_spec.BoundedArraySpec(shape=(len(stocks)+1), 
                                                       min=0, 
                                                       max=1, 
                                                       dtype=np.float32, 
                                                       name='action')
        self.observation_spec = array_spec.ArraySpec(shape=(number_of_feat),
                                                     dtype=np.float32)